## Import Required Packages

In [ ]:
import os
import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.optim import Adam

from pytorch_helper import *

## Global Configuration

In [ ]:
# Configure Device
use_cuda = True
device = torch.device("cuda" if torch.cuda.is_available() and use_cuda else "cpu")

# Setting manual seed for reproducability, also applies to gpu.
torch.manual_seed(1337)

## Training Configuration

In [ ]:
# Epochs and batch size
MAX_EPOCH = 1000
BATCH_SIZE = 300

# NN architecture
HIDDEN = 4
NEURONS = 150
INPUT = 6
OUTPUT = 256

# Learning rate schedule configuration
MAX_LR = 4e-2
START_LR = 0.001
END_LR = 2e-9
DIV_FACTOR = MAX_LR / START_LR
FINAL_DIV_FACTOR = MAX_LR / END_LR

# Regularization factor
REG_FACTOR = 0.00001

# DeepSHARQ delta
RI_RANGE = 0.1

## Create Log Paths

In [ ]:
# Create path for tensorboard log
tensorboard_path = "./runs/"
if not os.path.exists(tensorboard_path):
    os.makedirs(tensorboard_path)
    
# Create path to save models
models_path = "./models/"
if not os.path.exists(models_path):
    os.makedirs(models_path)

# Tensorboard setup ´tensorboard --logdir=runs´ -> http://localhost:6006/#scalars
log_name = "leakyReLU({},{})_K_range_{}_{}_max_lr={}_start_lr={}_final_lr={}_reg_factor={}".format(HIDDEN,NEURONS,RI_RANGE,MAX_EPOCH,MAX_LR,START_LR,END_LR,REG_FACTOR)
model_log_dir = models_path+log_name
tensorboard_log_dir = tensorboard_path+log_name
writer = SummaryWriter(log_dir=tensorboard_log_dir)

## Load Dataset

In [ ]:
# Get Train and Test datasets
train, validation, test = get_data_sets_k_minmax("datasets/k_range_{}_dataset.csv".format(RI_RANGE))

train_dl = DataLoader(dataset=train, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
validation_dl = DataLoader(dataset=validation, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)
test_dl = DataLoader(dataset=test, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

## Initialize Model and Optimizer

In [ ]:
model = DeepHEC(hidden_layers=HIDDEN, layer_size=NEURONS, inputs=INPUT, outputs=OUTPUT)
model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9, weight_decay=REG_FACTOR)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=MAX_LR, steps_per_epoch=len(train_dl), epochs=MAX_EPOCH, div_factor=DIV_FACTOR, final_div_factor=FINAL_DIV_FACTOR)

# for model inspection
print(model)

## Training Loop

In [ ]:
for epoch in range(1,MAX_EPOCH+1):
    model.train()
    train_loss = 0.
    lr = scheduler.get_last_lr()[0]
    
    # Run batch
    for batch, (X, y) in enumerate(train_dl):
        # Compute prediction and loss
        pred_k = model(X.to(device))
        loss = loss_fn(pred_k, y.to(device), device)
        train_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        
    # Compute train and validation loss
    train_loss = train_loss / float(len(train_dl))
    validation_loss, correct_k = validate_model_k_minmax(model, validation_dl, device)
    
    # Log data for Tensorboard
    print(f"Epoch {epoch}/{MAX_EPOCH}, t-loss={train_loss:>8f}, valid-acc-k:{(100*correct_k):>8f}%, v-loss={validation_loss:>8f}")
    writer.add_scalar("Loss/train", train_loss, epoch-1)
    writer.add_scalar("Loss/validation", validation_loss, epoch-1)
    writer.add_scalar("Accuracy(k)/validation", 100.*correct_k, epoch-1)
    writer.add_scalar("Scheduler/lr" ,lr, epoch-1)
    writer.flush()
    
# Save model
os.mkdir(model_log_dir)
torch.save(model.state_dict(),model_log_dir+"/model")
writer.close()